# December 03 - Parts 1 and 2

https://adventofcode.com/2018/day/2

In [ ]:
input_value = input("Please enter the puzzle text here, then press ENTER")

In [ ]:
# Having problems pasting multiple lines today... not sure why. Cheating a bit by splitting on the # character
# that appears on every line
input_words = input_value.split('#')
input_words = ["#"+x for x in input_words][1:]
input_words

In [ ]:
sample_input = [
    "#1 @ 1,3: 4x4",
    "#2 @ 3,1: 4x4",
    "#3 @ 5,5: 2x2"
]

In [ ]:
# Pick the list you want to work with
use_sample_input = False
if use_sample_input:
    words = sample_input
    dimensions = (10,10)
else:
    words = input_words
    dimensions = (1000,1000)


Today's problem is about figuring out how many "claims" to a 1000x1000 piece of fabric are overlapping. Each claim is provided in the puzzle input in the following format

`#{ID} {x},{y} {w}x{h}`

There is a handy format called Regular Expressions that are supported by most programming langagues. It allows you to define formats and sub-parts of a string that you can look for and extract. 

This is a very useful site for testing your regular expresions:

https://regexr.com/

In this case I'm going to create a format that matches and extracts the ID, x, y, w, h, and then return these in a dict object so they can easily be access later. 

In [ ]:
# We are going to use a regular expression to analyse the input
import regex as re
pattern = re.compile("^#(\d+) @ (\d+),(\d+): (\d+)x(\d+)")

def parse_line(line):
    matches = pattern.match(line)
    return dict(id=int(matches[1]),x=int(matches[2]),y=int(matches[3]),
                w=int(matches[4]),h=int(matches[5]),input=matches[0])

claims = [parse_line(x) for x in words]
claims[:5]

There are several ways we could go about solving this problem, but treating the whole piece of fabric as a large matrix would allow us to use some of the mathematical tools we use quite often, in this example a library called numpy.

http://www.numpy.org/

Numpy is very powerful for working with objects such as arrays and matrices. 

The sample data matches that in the problem description. It may be helpful to run through with the sample data first as you can easily see what's happening in each step.

In [ ]:
import numpy as np # We would normally place imports the top of the document, but they work inline too

# We can easily create arrays filled with zeros using numpy
np.zeros(100)

In [ ]:
# There are many ways of creating arrays
np.arange(100)

In [ ]:
# And we can reshape that array into a matrix of 10x10
np.arange(100).reshape(10,10)

In [ ]:
# Matrices can easily be sliced to select a subsection (we will use this below)
# Notice how the starting slice is inclusive, but the end one is exclusive - just like in lists
np.arange(100).reshape(10,10)[2:5,4:8] 

In [ ]:
# You can also do logical operations on matrices to get a new matrix filled with True and False
np.arange(100).reshape(10,10) >= 55

For this problem, we create a matrix full of zeros to represent the fabric

In [ ]:
# Create a matrix to hold our results
# We create a long list filled with zeros, and then use numpy to reshape it
matrix = np.zeros(dimensions[0] * dimensions[1]).reshape(dimensions[0],dimensions[1])
matrix

We now loop over each claim, select that part of the fabric matrix, and increment all the values.

In [ ]:
# Loop over each claim 
for claim in claims:    
    # Now create a slice of the main matrix and increment each cell
    matrix[claim["x"]:claim["x"]+claim["w"], claim["y"]:claim["y"]+claim["h"]] += 1

matrix

In [ ]:
# We now select only cells with a count larger than 1, and count the True values
# np.sum will treat False as zero and True as one, so we can simply sum to get the count
np.sum((matrix>1))

# Part 2

In part 2 we have to look for claims that don't overlap any others. Luckily we set this up to be really easy. Using our large matrix, we can loop over each claim and see if the slice of the large matrix that represents the claim only has ones in it. If it does, then only that claim covers that slice. 

In [ ]:
for claim in claims:
    # Create a slice having True if it contains a value larger other than one, or False otherwise
    # Count these values
    msum = np.sum(matrix[claim["x"]:claim["x"]+claim["w"], claim["y"]:claim["y"]+claim["h"]] > 1)
    
    # If the count is zero, then we only have ones (or zeros )
    if msum == 0:
        print(claim)
    